In [33]:
pip install crewai 'crewai[tools]' langchain_community

Note: you may need to restart the kernel to use updated packages.


In [34]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [35]:
from crewai import Agent, Task, Crew

In [36]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [37]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [38]:
financial_forecaster_agent = Agent(
    role="Financial Forecaster",
    goal="Develop and refine predictive models for asset pricing and risk assessment, providing accurate financial forecasts.",
    backstory="With a deep understanding of econometric techniques and financial theory, this agent crafts sophisticated models to anticipate market fluctuations. The Financial Forecaster Agent is essential for strategic planning and investment optimization, leveraging historical data and current market conditions to project future outcomes.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

In [39]:
risk_manager_agent = Agent(
    role="Risk Manager",
    goal="Identify, assess, and mitigate financial risks across various portfolios using advanced quantitative techniques.",
    backstory="With a keen eye for potential vulnerabilities, this agent excels in building robust risk models and conducting stress tests. The Risk Manager Agent is crucial for safeguarding investments and ensuring regulatory compliance, employing sophisticated calculations and data analysis to minimize exposure to adverse market conditions.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

In [40]:
portfolio_optimizer_agent = Agent(
    role="Portfolio Optimizer",
    goal="Construct and rebalance investment portfolios to maximize returns while adhering to specific risk tolerances and investment objectives.",
    backstory="This agent specializes in applying optimization algorithms and financial modeling to create efficient portfolios. The Portfolio Optimizer Agent is essential for achieving optimal investment outcomes, leveraging data analysis and complex calculations to dynamically adjust asset allocations in response to changing market dynamics.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

In [41]:
financial_forecast_task = Task(
    description=(
        "Quick forecast for {stock_selection} price next quarter. Focus on key metrics only."
    ),
    expected_output=(
        "Brief price forecast with key support/resistance levels for {stock_selection}."
    ),
    agent=financial_forecaster_agent,
    timeout=120  # Reduced to 2 minutes
)

In [42]:
risk_assessment_task = Task(
    description=(
        "Quick risk assessment for {stock_selection}. List main risk factors."
    ),
    expected_output=(
        "Key risks and basic mitigation steps for {stock_selection}."
    ),
    agent=risk_manager_agent,
    timeout=120
)

In [43]:
portfolio_optimization_task = Task(
    description=(
        "Basic portfolio allocation for {stock_selection} with {initial_capital} and {risk_tolerance} risk."
    ),
    expected_output=(
        "Simple allocation strategy for {stock_selection} considering {risk_tolerance} risk."
    ),
    agent=portfolio_optimizer_agent,
    timeout=120
)

In [44]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_analysis_crew = Crew(
    agents=[financial_forecaster_agent, 
            risk_manager_agent, 
            portfolio_optimizer_agent],
    
    tasks=[financial_forecast_task, 
           risk_assessment_task, 
           portfolio_optimization_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.3),
    process=Process.hierarchical,
    verbose=True
)

2025-03-09 15:40:41,833 - 8674969664 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [45]:
financial_analysis_inputs = {
    'stock_selection': 'CAP.PA',
    'initial_capital': '250000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [46]:
result = financial_analysis_crew.kickoff(inputs=financial_analysis_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Quick forecast for CAP.PA price next quarter. Focus on key metrics only.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of creating a quick forecast for CAP.PA price next quarter to the Financial Forecaster. They have the expertise to analyze key metrics and provide support/resistance levels.

Action: Delegate work to co-worker
Action Input: {"coworker": "Financial Forecaster", "task": "Create a quick forecast for CAP.PA price next quarter focusing on key metrics and support/resistance levels", "context": "The forecast should be based on historical data, market trends, and any relevant news impacting CAP.PA"}


> Entering new CrewAgentExecutor chain...
I should start by gathering historical data on CAP.PA, analyzing market trends, and searching for any relevant news that could impact its price next quarter.

Action: Search the internet
Action Input: {"search_query": "CAP.PA historical stock data"} 



In [47]:
from IPython.display import Markdown
Markdown(result)

Factors influencing severity of community-acquired pneumonia include prior respiratory infection, obesity, alcoholism, and old age (>60 years). Age, smoking, environmental exposures, malnutrition, previous CAP, chronic bronchitis/chronic obstructive pulmonary disease, asthma, and functional impairment are also significant risk factors. In addition, comorbid factors such as underlying cardiopulmonary disease, diminished splenic function, HIV, COPD, asthma, smoking, low social status, and various comorbidities can increase the risk of severe CAP. Modifiable behavioral risk factors like smoking, alcohol overuse, and opioid use should also be considered in the context of CAP risk assessment.